<a href="https://colab.research.google.com/github/ModupalliParimala/Music_Genres_Classification/blob/main/Music_Genres_classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Deep_Learning/Projects/

/content/gdrive/MyDrive/Deep_Learning/Projects


In [3]:
%tensorflow_version 2.x

In [4]:
import pandas as pd
import numpy as np
from numpy import argmax
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization,Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model,model_from_json
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

In [ ]:
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    os.makedirs(os.path.join("/content/gdrive/MyDrive/Deep_Learning/Projects",f'img_data/{g}'),exist_ok=True)
    for filename in os.listdir(f'./genres/{g}'):
        song = f'./genres/{g}/{filename}'
        y,sr = librosa.load(song,sr=None)
        mels = librosa.feature.melspectrogram(y=y,sr=sr)
        fig = plt.Figure()
        canvas = FigureCanvas(fig)
        librosa.display.specshow(librosa.power_to_db(mels,ref=np.max))
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.jpg')
        plt.close()    
        fig.clf()
        plt.close(fig)
        plt.close('all')
    
del g,filename,song,y,sr,fig,canvas

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders 
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio('./img_data/', output="./data", seed=1337, ratio=(.8, .2)) # default value

Copying files: 1000 files [04:45,  3.50 files/s]


In [5]:
#Data augmentation is not required in case spectrogram.just standardizing image data by dividing by 255
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255 # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
        ) 
test_datagen = ImageDataGenerator(
        rescale=1./255 # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
        ) 

In [6]:
training_set = train_datagen.flow_from_directory(
        './data/train',
        target_size=(64, 64),
        batch_size=40,
        class_mode='categorical')
test_set = test_datagen.flow_from_directory(
        './data/val',
        target_size=(64, 64),
        batch_size=40,
        class_mode='categorical' )

Found 800 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [7]:
input_shape=(64, 64, 3)

model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same',
                 input_shape=input_shape,
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3),padding='same',kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same',kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3),padding='same',kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3),padding='same',kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(512, (3, 3),padding='same',kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(10, activation='softmax'))


In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
outputFolder="./output"
# Model weights are saved at the end of every epoch, if it's the best seen so far.
filepath=outputFolder+"/weights-{epoch:02d}-{accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=True, 
                             mode='auto') 


In [ ]:
#Early Stopping
earlystop = EarlyStopping(monitor='loss', 
                          min_delta=0.01, patience=5,
                          verbose=1, mode='auto')

In [12]:
# if the accuracy does not increase over 10 epochs, reduce the learning rate by half.
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
model.fit(
        training_set,
        epochs=200,
        verbose = 1,
        steps_per_epoch=training_set.n//training_set.batch_size,
        validation_data=test_set,
        validation_steps=test_set.n//test_set.batch_size,
        callbacks=[reduce_lr,checkpoint])

Epoch 1/200
20/20 [==============================] - 4s 200ms/step - loss: 2.1483 - accuracy: 0.3113 - val_loss: 3.4035 - val_accuracy: 0.1000

Epoch 00001: accuracy improved from 0.28625 to 0.31125, saving model to ./output/weights-01-0.3113.h5
Epoch 2/200
20/20 [==============================] - 4s 200ms/step - loss: 1.9045 - accuracy: 0.3887 - val_loss: 3.7606 - val_accuracy: 0.1000

Epoch 00002: accuracy improved from 0.31125 to 0.38875, saving model to ./output/weights-02-0.3887.h5
Epoch 3/200
20/20 [==============================] - 4s 202ms/step - loss: 1.7754 - accuracy: 0.4100 - val_loss: 3.9702 - val_accuracy: 0.1100

Epoch 00003: accuracy improved from 0.38875 to 0.41000, saving model to ./output/weights-03-0.4100.h5
Epoch 4/200
20/20 [==============================] - 4s 202ms/step - loss: 1.7904 - accuracy: 0.4125 - val_loss: 4.5756 - val_accuracy: 0.1400

Epoch 00004: accuracy improved from 0.41000 to 0.41250, saving model to ./output/weights-04-0.4125.h5
Epoch 5/200
20/2

In [17]:
model.load_weights("././output/weights-166-0.9975.h5")# Got 99.75 accuracy on training data

In [18]:
#Model Evaluation
model.evaluate(test_set,
        steps=test_set.n//test_set.batch_size,verbose=1)#OUTPUT


5/5 [==============================] - 1s 152ms/step - loss: 1.7540 - accuracy: 0.6050


[1.7539807558059692, 0.6050000190734863]

In [19]:

pred = model.predict(test_set, steps=test_set.n//test_set.batch_size, verbose=1)

5/5 [==============================] - 1s 147ms/step


In [20]:
predicted_class_indices=np.argmax(pred,axis=1)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:200]
filenames=test_set.filenames

In [21]:
print(len(filenames), len(predictions))
# (200, 200)

200 200


In [24]:
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("Music_Genres_prediction_CNN.csv",index=False)